In [1]:
import ee
import collections
collections.Callable = collections.abc.Callable
import geemap
from geemap import chart

%matplotlib inline
import matplotlib.pyplot as plt

ee.Authenticate()
ee.Initialize()

Enter verification code:  4/1AX4XfWioQpEEx5V9x_pSThfkO7Xco6W6eUBMvmeeLmGqYuAo4f8RpSljM_0



Successfully saved authorization token.


## Imports and Constants

##### test areas, data sources, composite RGB image

In [82]:
import ee
import geemap

Map = geemap.Map()

area1 = ee.Geometry.Polygon(
        [[[25.75, -10.75],
          [25.75, -10.8],
          [25.9, -10.8],
          [25.9, -10.75]]], None, False)
area2 = ee.Geometry.Polygon(
        [[[25.8, -10.6],
          [25.8, -10.7],
          [25.95, -10.7],
          [25.95, -10.6]]], None, False)
area3 = ee.Geometry.Polygon(
        [[[26, -10.5],
          [26, -10.75],
          [26.25, -10.75],
          [26.25, -10.5]]], None, False)
area4 = ee.Geometry.Polygon(
        [[[25.95, -10.6],
          [25.95, -10.7],
          [26.05, -10.7],
          [26.05, -10.6]]], None, False)
noMine = ee.Geometry.Polygon(
        [[[26.05, -10.7],
          [26.05, -10.8],
          [26.15, -10.8],
          [26.15, -10.7]]], None, False)
rishiArea = ee.Geometry.Polygon(
        [[[27.35, -7.5],
          [27.35, -7.6],
          [27.45, -7.6],
          [27.45, -7.5]]], None, False)
ls8 = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")
s2 = ee.ImageCollection("COPERNICUS/S2_SR")
s1 = ee.ImageCollection('COPERNICUS/S1_GRD')
modis = ee.ImageCollection("MODIS/006/MOD13Q1") # vegetation indices
modis2 = ee.ImageCollection('MODIS/006/MCD43A4')
rgbVis = {
    'min': 0,
    'max': 3000,
    'bands': ['B4', 'B3', 'B2']
}
comp2020 = s2 \
            .filter(ee.Filter.bounds(area2)) \
            .filter(ee.Filter.date('2020-01-01', '2020-12-31')) \
            .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
            .select('B.*') \
            .median().clip(area2)
sar2020 = s1 \
            .filter(ee.Filter.eq('instrumentMode','IW')) \
            .filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING')) \
            .filter(ee.Filter.eq('resolution_meters',10)) \
            .filter(ee.Filter.intersects('.geo', area2)) \
            .filter(ee.Filter.date('2020-01-01', '2020-12-31')) \
            .select(['VV', 'VH']).median().clip(area2)

### Create .05x.05 Regions

In [83]:
def divide_geometry():
    regions = []
    i = 25.8
    while i <=25.95:
        j = -10.6
        while j >= -10.7:
            region = ee.Geometry.Polygon(
                [[[i, j],
                  [i, j-.05],
                  [i+.05, j-.05],
                  [i+.05, j]]])
            regions.append((region))
            j = j-.05
        i = i+.05
    return regions

In [84]:
regions = divide_geometry()
print('Number of Regions Created: ' + str(len(regions)))

Number of Regions Created: 6


### NDVI: Normalized Difference Vegetation Index

##### Using the pre-calculated NDVI band from Modis data. Values range from -2000 to 10000
##### NDVI is generally lower in mining areas, and decreases as mining increases

##### Function to generate NDVI images from Modis Data for a given start and end year

In [85]:
def create_ndvi_layers_modis(start, end, geometry):
    images = []
    ndviStart = modis \
                .filter(ee.Filter.bounds(geometry))\
                .filter(ee.Filter.date(str(start) + '-01-01', str(start) + '-12-31')) \
                .select('NDVI') \
                .median() \
                .clip(geometry)
    images.append((ndviStart, start))
    ndviEnd = modis \
                .filter(ee.Filter.bounds(geometry))\
                .filter(ee.Filter.date(str(end) + '-01-01', str(end) + '-12-31')) \
                .select('NDVI') \
                .median() \
                .clip(geometry)
    images.append((ndviEnd, end))
    return images

##### Function to generate an average NDVI value for a given region and year

In [86]:
def extract_ndvi_modis(image):
    stats = image.reduceRegion(**{
        'reducer': ee.Reducer.mean(),
        'geometry': image.geometry(),
        'scale': 100
    })
    
    properties = {
        'Year': image.get('system:index'),
        'NDVI': stats.get('NDVI')
    }
    
    return ee.Feature(None, properties)

##### Function to determine if the NDVI loss of a given region over given years is significant enough to be a possible mining location

In [87]:
def ndvi_requirement_modis(df):
    status = 0 # 1 = possible mine, 0 = not mine
    if df.iat[0,1] < 4000:
        status = 1
    elif df.iat[0,1]-df.iat[1,1] > 1000:
        status = 1
    else: 
        status = status
    return status

##### Function to determine what regions within an area may be possible mine locations based on NDVI, and add these regions to a map as composite images 

In [88]:
def ndvi_level_modis(regions):
    Map.centerObject(area2, 11)
    Map.addLayer(comp2020,rgbVis,'whole',False)
    num = 1
    for region in regions:
        layerName = str(num) + ' ndvi pass'
        
        images = ee.ImageCollection(create_ndvi_layers_modis(2004, 2020, region))
        data = ee.FeatureCollection(images.map(extract_ndvi_modis))
        df = geemap.ee_to_pandas(data)

        status = ndvi_requirement_modis(df)
        if status == 1:
            image = s2 \
                .filter(ee.Filter.bounds(region)) \
                .filter(ee.Filter.date('2020-01-01', '2020-12-31')) \
                .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
                .select('B.*') \
                .median().clip(region)
            Map.addLayer(image, rgbVis, layerName)
        num = num+1
    return Map

##### Calls the above function on the selected regions

In [89]:
ndvi_level_modis(regions)

Map(center=[-10.650000238244901, 25.875000000000533], controls=(WidgetControl(options=['position', 'transparen…

### Iron Index

##### RED = band 1, BLUE = band 3
##### Use normalized difference
##### Iron is generally higher in mining areas and increases as mining increases. However, there can be similar values with roads and urban areas. 

##### Function to generate Iron images from Modis Data for a given start and end year

In [12]:
def create_iron_layers_modis(start, end, geometry):
    images = []
    ironStart = modis2 \
                .filter(ee.Filter.bounds(geometry))\
                .filter(ee.Filter.date(str(start) + '-01-01', str(start) + '-12-31')) \
                .median() \
                .clip(geometry)
    ironStart2 = ironStart.normalizedDifference(['Nadir_Reflectance_Band1', 'Nadir_Reflectance_Band3']).rename('IRON')
    images.append((ironStart2, start))
    ironEnd = modis2 \
                .filter(ee.Filter.bounds(geometry))\
                .filter(ee.Filter.date(str(end) + '-01-01', str(end) + '-12-31')) \
                .median() \
                .clip(geometry)
    ironEnd2 = ironEnd.normalizedDifference(['Nadir_Reflectance_Band1', 'Nadir_Reflectance_Band3']).rename('IRON')
    images.append((ironEnd2, end))
    return images

##### Function to generate an average Iron value for a given region and year

In [13]:
def extract_iron_modis(image):
    stats = image.reduceRegion(**{
        'reducer': ee.Reducer.mean(),
        'geometry': image.geometry(),
        'scale': 100
    })
    
    properties = {
        'Year': image.get('system:index'),
        'IRON': stats.get('IRON')
    }
    
    return ee.Feature(None, properties)

##### Function to determine if the Iron gain of a given region over given years is significant enough to be a possible mining location

In [15]:
def iron_requirement_modis(df):
    status = 0 # 1 = possible mine, 0 = not mine
    if df.iat[0,1] > .35: # must be greater than .3 # .35 # might need to increase slightly
        status = 1
    elif df.iat[1,1]-df.iat[0,1] > .035: #.035
        status = 1
    else: 
        status = status
    return status

##### Function to determine what regions within an area may be possible mine locations based on Iron, and add these regions to a map as composite images 

In [16]:
def iron_level_modis(regions):
    Map2 = geemap.Map()
    Map2.centerObject(area1, 11)
    Map2.addLayer(comp2020,rgbVis,'whole',False)
    num = 1
    for region in regions:
        layerName = str(num) + ' iron pass'
        
        images = ee.ImageCollection(create_iron_layers_modis(2004, 2020, region))
        data = ee.FeatureCollection(images.map(extract_iron_modis))
        df = geemap.ee_to_pandas(data)

        status = iron_requirement_modis(df)
        if status == 1:
            image = s2 \
                .filter(ee.Filter.bounds(region)) \
                .filter(ee.Filter.date('2020-01-01', '2020-12-31')) \
                .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
                .select('B.*') \
                .median().clip(region)
            Map2.addLayer(image, rgbVis, layerName)
        num = num+1
    return Map2

##### Calls the above function on the selected regions

In [33]:
iron_level_modis(regions)

### NDMI: Normalized Difference Moisture Index

##### NIR = band 2, SWIR1 = band 6
##### NDMI is generally negative in mining areas, and decreases as mining increases

##### Function to generate NDMI images from Modis Data for a given start and end year

In [90]:
def create_ndmi_layers_modis(start, end, geometry):
    images = []
    ndmiStart = modis2 \
                .filter(ee.Filter.bounds(geometry))\
                .filter(ee.Filter.date(str(start) + '-01-01', str(start) + '-12-31')) \
                .median() \
                .clip(geometry)
    ndmiStart2 = ndmiStart.normalizedDifference(['Nadir_Reflectance_Band2', 'Nadir_Reflectance_Band6']).rename('NDMI')
    images.append((ndmiStart2, start))
    ndmiEnd = modis2 \
                .filter(ee.Filter.bounds(geometry))\
                .filter(ee.Filter.date(str(end) + '-01-01', str(end) + '-12-31')) \
                .median() \
                .clip(geometry)
    ndmiEnd2 = ndmiEnd.normalizedDifference(['Nadir_Reflectance_Band2', 'Nadir_Reflectance_Band6']).rename('NDMI')
    images.append((ndmiEnd2, end))
    return images

##### Function to generate an average NDMI value for a given region and year

In [91]:
def extract_ndmi_modis(image):
    stats = image.reduceRegion(**{
        'reducer': ee.Reducer.mean(),
        'geometry': image.geometry(),
        'scale': 100
    })
    
    properties = {
        'Year': image.get('system:index'),
        'NDMI': stats.get('NDMI')
    }
    
    return ee.Feature(None, properties)

##### Function to determine if the NDMI loss of a given region over given years is significant enough to be a possible mining location

In [92]:
def ndmi_requirement_modis(df):
    status = 0 # 1 = possible mine, 0 = not mine
    if df.iat[0,1] < 0: #goes negative
        status = 1
    elif df.iat[0,1]-df.iat[1,1] > .1: 
        status = 1
    else: 
        status = status
    return status

##### Function to determine what regions within an area may be possible mine locations based on NDMI, and add these regions to a map as composite images 

In [93]:
def ndmi_level_modis(regions):
    Map3 = geemap.Map()
    Map3.centerObject(area2, 11)
    Map3.addLayer(comp2020,rgbVis,'whole',False)
    num = 1
    for region in regions:
        layerName = str(num) + ' ndmi pass'
        
        images = ee.ImageCollection(create_ndmi_layers_modis(2004, 2020, region))
        data = ee.FeatureCollection(images.map(extract_ndmi_modis))
        df = geemap.ee_to_pandas(data)

        status = ndmi_requirement_modis(df)
        if status == 1:
            image = s2 \
                .filter(ee.Filter.bounds(region)) \
                .filter(ee.Filter.date('2020-01-01', '2020-12-31')) \
                .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
                .select('B.*') \
                .median().clip(region)
            Map3.addLayer(image, rgbVis, layerName)
        num = num+1
    return Map3

##### Calls the above function on the selected regions

In [94]:
ndmi_level_modis(regions)

Map(center=[-10.650000238244901, 25.875000000000533], controls=(WidgetControl(options=['position', 'transparen…

### NIR/SWIR2

##### NIR = band 2, SWIR2 = band 7
##### Use normalized difference
##### NIR/SWIR2 is significantly lower in mining areas, and decreases as mining increases

##### Function to generate NIR/SWIR2 images from Modis Data for a given start and end year

In [95]:
def create_nirSwir2_layers_modis(start, end, geometry):
    images = []
    nirSwir2Start = modis2 \
                .filter(ee.Filter.bounds(geometry))\
                .filter(ee.Filter.date(str(start) + '-01-01', str(start) + '-12-31')) \
                .median() \
                .clip(geometry)
    nirSwir2Start2 = nirSwir2Start.normalizedDifference(['Nadir_Reflectance_Band2', 'Nadir_Reflectance_Band7']).rename('nirSwir2')
    images.append((nirSwir2Start2, start))
    nirSwir2End = modis2 \
                .filter(ee.Filter.bounds(geometry))\
                .filter(ee.Filter.date(str(end) + '-01-01', str(end) + '-12-31')) \
                .median() \
                .clip(geometry)
    nirSwir2End2 = nirSwir2End.normalizedDifference(['Nadir_Reflectance_Band2', 'Nadir_Reflectance_Band7']).rename('nirSwir2')
    images.append((nirSwir2End2, end))
    return images

##### Function to generate an average NIR/SWIR2 value for a given region and year

In [96]:
def extract_nirSwir2_modis(image):
    stats = image.reduceRegion(**{
        'reducer': ee.Reducer.mean(),
        'geometry': image.geometry(),
        'scale': 100
    })
    
    properties = {
        'Year': image.get('system:index'),
        'nirSwir2': stats.get('nirSwir2')
    }
    
    return ee.Feature(None, properties)

##### Function to determine if the NIR/SWIR2 loss of a given region over given years is significant enough to be a possible mining location

In [97]:
def nirSwir2_requirement_modis(df):
    status = 0 # 1 = possible mine, 0 = not mine
    if df.iat[0,1] < .4: 
        status = 1
    elif df.iat[0,1]-df.iat[1,1] > .25: 
        status = 1
    else: 
        status = status
    return status

##### Function to determine what regions within an area may be possible mine locations based on NIR/SWIR2, and adds these regions to a map as composite images 

In [98]:
def nirSwir2_level_modis(regions):
    Map4 = geemap.Map()
    Map4.centerObject(area2, 11)
    Map4.addLayer(comp2020,rgbVis,'whole',False)
    num = 1
    for region in regions:
        layerName = str(num) + ' nir/Swir2 pass'
        
        images = ee.ImageCollection(create_nirSwir2_layers_modis(2004, 2020, region))
        data = ee.FeatureCollection(images.map(extract_nirSwir2_modis))
        df = geemap.ee_to_pandas(data)

        status = nirSwir2_requirement_modis(df)
        if status == 1:
            image = s2 \
                .filter(ee.Filter.bounds(region)) \
                .filter(ee.Filter.date('2020-01-01', '2020-12-31')) \
                .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
                .select('B.*') \
                .median().clip(region)
            Map4.addLayer(image, rgbVis, layerName)
        num = num+1
    return Map4

##### Calls the above function on the selected regions

In [99]:
nirSwir2_level_modis(regions)

Map(center=[-10.650000238244901, 25.875000000000533], controls=(WidgetControl(options=['position', 'transparen…

### Water Storage 2020

In [29]:
ndvi = comp2020.normalizedDifference(['B8', 'B4'])
nirG = comp2020.normalizedDifference(['B8', 'B3'])
ndmi = comp2020.normalizedDifference(['B8', 'B11'])
        
vv2020 = sar2020.select('VV')
        
waterStorage = vv2020.lt(-15).rename('water').And(nirG.lt(0.1).Or(ndmi.gt(.3))).selfMask()
connectW = waterStorage.connectedPixelCount(25)
waterStorage = waterStorage.updateMask(connectW.gt(8))
Map.addLayer(waterStorage, {'min':0, 'max':1, 'palette':['blue']}, 'Water')
Map

Map(bottom=278230.0, center=[-10.775002314558064, 25.825000000002213], controls=(WidgetControl(options=['posit…